<a href="https://colab.research.google.com/github/matheusBernal/GymExercises/blob/main/GymExercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project uses data from gym members' exercise routines, physical attributes, and other fitness metrics. I will use this information to create two algorithms: one to classify the experience level and workout type, and another to perform regression to predict the calories burned.

In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install sklearn
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score,mean_squared_error,mean_absolute_error
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv('/content/gym_members_exercise_tracking.csv')
df

In [ ]:
df = df[['Age', 'Gender', 'Weight (kg)', 'Height (m)', 'Max_BPM', 'Avg_BPM','Resting_BPM', 'Session_Duration (hours)','Fat_Percentage', 'Water_Intake (liters)','Workout_Frequency (days/week)', 'BMI','Calories_Burned','Experience_Level','Workout_Type']]

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe()

I'm going to divide my data into three groups: workout type, experience level, and calories burned. First, I will create the regression model for calories burned. After that, I will create the classification for experience level, and finally, I will create the classification for workout type.

In [ ]:
 #First i'll need to using the label encoder on column Gender to pass on train regression
genderLabel = LabelEncoder()
df_preprocessing = df
df_preprocessing['Gender'] = genderLabel.fit_transform(df_preprocessing['Gender'])
df_preprocessing

Now, let's examine the correlation between the variables and calories burned and select the strongest variables (positive or negative) to create our regression model. If the variable is greater than 0.5, it is strongly positive (variable > 0.5). If the variable is less than -0.5, it is strongly negative (variable < 0.5). And if the variable is close to or equal to zero, its strength is considered null.

In [ ]:
df_preprocessing.iloc[:,:-2].corr()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df_preprocessing.iloc[:,:-2].corr(),annot=True)

In [ ]:
x_cb = df_preprocessing[['Session_Duration (hours)','Fat_Percentage','Workout_Frequency (days/week)']].values
y_cb = df_preprocessing['Calories_Burned'].values

In [ ]:
x_cb_train,x_cb_test,y_cb_train,y_cb_test = train_test_split(x_cb,y_cb,test_size=0.2,random_state=0)

In [ ]:
model = LinearRegression()
linear_model = model.fit(x_cb_train,y_cb_train)
prediction_train = linear_model.predict(x_cb_train)
prediction_test = linear_model.predict(x_cb_test)
print(linear_model.coef_,linear_model.intercept_)
print('Train score:',round(linear_model.score(x_cb_train,y_cb_train),3)*100)
print('Test score:',round(linear_model.score(x_cb_test,y_cb_test),3)*100)
print(mean_squared_error(y_cb_train,prediction_train),mean_squared_error(y_cb_test,prediction_test))
print(mean_absolute_error(y_cb_train,prediction_train),mean_squared_error(y_cb_test,prediction_test))

The function that our LineaRegression create is Y = 694.51683961 * X  -4.95866653 * Z -13.94390992 * W + 203.9242449799383. And the score on the train's data is 83% and tests data is 84,5%

In [ ]:
sns.scatterplot(x=x_cb_train[:, 0], y=y_cb_train)
sns.scatterplot(x=x_cb_train[:, 0], y=prediction_train)
plt.legend(['Train','Train prediction'])
plt.xlabel('Session_Duration (hours)')
plt.ylabel('Calories Burned')
plt.title('Calories Burned vs. Session Duration (hours)')
plt.show()

In [ ]:
sns.scatterplot(x=x_cb_train[:, 1], y=y_cb_train)
sns.scatterplot(x=x_cb_train[:, 1], y=prediction_train)
plt.legend(['Train','Train prediction'])
plt.xlabel('Fat Percentage')
plt.ylabel('Calories Burned')
plt.title('Calories Burned vs. Fat_Percentage')
plt.show()

In [ ]:
sns.scatterplot(x=x_cb_train[:, 2], y=y_cb_train)
sns.scatterplot(x=x_cb_train[:, 2], y=prediction_train)
plt.legend(['Train','Train prediction'])
plt.xlabel('Workout_Frequency (days/week)')
plt.ylabel('Calories Burned')
plt.title('Calories Burned vs. Workout_Frequency (days/week)')
plt.show()

In [ ]:
sns.scatterplot(x=x_cb_test[:, 0], y=y_cb_test)
sns.scatterplot(x=x_cb_test[:, 0], y=prediction_test)
plt.legend(['Train','Train prediction'])
plt.xlabel('Session_Duration (hours)')
plt.ylabel('Calories Burned')
plt.title('Calories Burned vs. Session Duration (hours)')
plt.show()


In [ ]:
sns.scatterplot(x=x_cb_test[:, 1], y=y_cb_test)
sns.scatterplot(x=x_cb_test[:, 1], y=prediction_test)
plt.legend(['Test','Test prediction'])
plt.xlabel('Fat Percentage')
plt.ylabel('Calories Burned')
plt.title('Calories Burned vs. Fat_Percentage')
plt.show()

In [ ]:
sns.scatterplot(x=x_cb_test[:, 2], y=y_cb_test)
sns.scatterplot(x=x_cb_test[:, 2], y=prediction_test)
plt.legend(['Test','Test prediction'])
plt.xlabel('Workout_Frequency (days/week)')
plt.ylabel('Calories Burned')
plt.title('Calories Burned vs. Workout_Frequency (days/week)')
plt.show()

Let's try with Ridge function

In [ ]:
ridge = Ridge()
ridge_model = ridge.fit(x_cb_train,y_cb_train)
prediction_train = ridge_model.predict(x_cb_train)
prediction_test = ridge_model.predict(x_cb_test)
print(ridge_model.coef_,ridge_model.intercept_)
print('Train score:',round(ridge_model.score(x_cb_train,y_cb_train),3)*100)
print('Test score:',round(ridge_model.score(x_cb_test,y_cb_test),3)*100)


Th function that our Ridge create is Y = 679.85919549 * X -5.22780541 * Z -11.40942128 * W + 220.70894588408976. And the score on the train's data is 83% and tests data is 84,5%. So we don't have any difference between the Ridge and the Linear Model in fact

Now let's make the Experience Level classification

In [ ]:
df_preprocessing.shape

In [ ]:
#First, I will use the OneHotEncoder and ColumnTransformer on the Gender column. Finally, I will apply StandardScaler to prepare the data for the classification model.
OneHotEncoderGender = ColumnTransformer([('Gender',OneHotEncoder(),[1])],remainder='passthrough')
x_experiênce_level = df_preprocessing.iloc[:,:-2].values
y_experience_level = df_preprocessing.iloc[:,-2].values
x_experiênce_level = OneHotEncoderGender.fit_transform(x_experiênce_level)
x_experiênce_level = StandardScaler().fit_transform(x_experiênce_level)
x_experiênce_level.shape

Now Let's try using some methods to create our classification. First let's try use Gaussian. Accuracy 88%

In [ ]:
gaussianModel = GaussianNB()
x_experiênce_level_train,x_experiênce_level_test,y_experience_level_train,y_experience_level_test = train_test_split(x_experiênce_level,y_experience_level,test_size=0.2,random_state=0)
gaussian_model = gaussianModel.fit(x_experiênce_level_train,y_experience_level_train)

In [ ]:
prediction_train_level = gaussian_model.predict(x_experiênce_level_train)
prediction_test_level = gaussian_model.predict(x_experiênce_level_test)

print('Train score:',round(accuracy_score(y_experience_level_train,prediction_train_level),3)*100)
print('Test score:',round(accuracy_score(y_experience_level_test,prediction_test_level),3)*100)

In [ ]:
confusion_matrix(y_experience_level_train,prediction_train_level),confusion_matrix(y_experience_level_test,prediction_test_level)

In [ ]:
print(classification_report(y_experience_level_train,prediction_train_level),classification_report(y_experience_level_test,prediction_test_level))

In [ ]:
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

grid_search = GridSearchCV(estimator=gaussianModel, param_grid=param_grid, cv=5)
grid_search.fit(x_experiênce_level_train, y_experience_level_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Now Let's use the Decision Tree and forward I'll use the grid Search to create the algorithm. Accuracy: 87%

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}
grid_search = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_experiênce_level_train, y_experience_level_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)

In [ ]:
decisionTreeXpLevel = DecisionTreeClassifier(criterion = 'gini', max_depth = 30, max_features= None, min_samples_leaf = 1, min_samples_split= 10)
decisionTreeXpLevel_model = decisionTreeXpLevel.fit(x_experiênce_level_train,y_experience_level_train)
print('Train score: ',accuracy_score(y_experience_level_train,decisionTreeXpLevel_model.predict(x_experiênce_level_train)))
print('Test score: ',accuracy_score(y_experience_level_test,decisionTreeXpLevel_model.predict(x_experiênce_level_test)))

Let's see the tree

In [ ]:
figure,ax = plt.subplots(nrows=1,ncols=1,figsize=(10,10))
tree.plot_tree(
    decisionTreeXpLevel_model,
    filled=True,
    ax=ax
)

In [ ]:
confusion_matrix(y_experience_level_train,decisionTreeXpLevel_model.predict(x_experiênce_level_train)),confusion_matrix(y_experience_level_test,decisionTreeXpLevel_model.predict(x_experiênce_level_test))

In [ ]:
print(classification_report(y_experience_level_train,decisionTreeXpLevel_model.predict(x_experiênce_level_train)),classification_report(y_experience_level_test,decisionTreeXpLevel_model.predict(x_experiênce_level_test)))

Now I'm going to use the random forest algoritm.Accuracy

In [ ]:
param_grid = {
    'n_estimators': [100, 150, 200],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': [None, 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'oob_score': [True, False]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_experiênce_level_train, y_experience_level_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)